### GP Sarpanch Level Output

In [1]:
import pandas as pd
from functools import reduce
import os
import zipfile

### Load dat

In [2]:
up_2005 = pd.read_csv("../data/up_gp_sarpanch_2005_fixed.csv", low_memory = False)
up_2010 = pd.read_csv("../data/up_gp_sarpanch_2010_fixed.csv", low_memory = False)

In [12]:
# 2015
dataframes = []
folder_path = "../data/2015"

for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv') and 'ग्राम पंचायत प्रधान' in file_name:
        file_path = os.path.join(folder_path, file_name)
        df = pd.read_csv(file_path)
        df['district_name'] = file_name.split('-ग्राम पंचायत प्रधान')[0]
        dataframes.append(df)

up_2015 = pd.concat(dataframes, ignore_index=True)

column_transliteration_map = {
    'ब्लॉक': 'block',
    'ग्राम पंचायत': 'gp',
    'पद का आरक्षण': 'gp_reservation_status',
    'उम्मीदवार': 'elected_sarpanch_name',
    'पिता/पति': 'father_husband',
    'प्रत्याशी का आरक्षण': 'candidate_reservation',
    'शैक्षिक योग्यता': 'educational_qualification',
    'लिंग': 'sex',
    'मोबाइल नं०': 'mobile_number',
    'प्राप्त वैध मत': 'valid_votes_received',
    'प्राप्त मत %': 'votes_received_percent',
    'मतदान %': 'voting_percent',
    'परिणाम': 'result',
    'जिला पंचायत': 'district_panchayat',
    'क्षेत्र पंचायत वार्ड': 'area_panchayat_ward',
    'जिला पंचायत वार्ड': 'district_panchayat_ward',
    'जिला': 'district_name',
    'क्षेत्र पंचायत': 'area_panchayat'
}

up_2015 = up_2015.rename(columns=column_transliteration_map)

In [13]:
up_2015['block'] = up_2015['block'].fillna('')
up_2015['block'] = up_2015['block'].apply(lambda x: x if ' - ' in x else f'0 - {x}')
up_2015[['block_num', 'block_name']] = up_2015['block'].str.split(' - ', n=1, expand=True)

In [14]:
up_2015['gp'] = up_2015['gp'].fillna('')
up_2015['gp'] = up_2015['gp'].apply(lambda x: x if ' - ' in x else f'0 - {x}')
up_2015[['gp_num', 'gp_name']] = up_2015['gp'].str.split(' - ', n=1, expand=True)
up_2015['gp_num'] = up_2015['gp_num'].str.strip()
up_2015['gp_name'] = up_2015['gp_name'].str.strip()

In [15]:
# 2021
with zipfile.ZipFile("../data/up_gram_panchayat_pradhan_2021.csv.zip", 'r') as zip_ref:
    csv_file = [f for f in zip_ref.namelist() if f.endswith('.csv') and not f.startswith('__MACOSX')]
    
    with zip_ref.open(csv_file[0]) as a_csv_file:
        up_2021 = pd.read_csv(a_csv_file, encoding='utf-8')

column_transliteration_map = {
    'zila': 'district_name',
    'block': 'block_name',
    'candidate_name_2021': 'candidate',
    'father_husband_name_2021': 'father_husband',
    'gram_panchayat': 'gp',
    'gender_2021': 'sex',
    'age_2021': 'age',
    'education_2021': 'education',
    'caste_2021': 'candidate_res_status',
}

up_2021 = up_2021.rename(columns=column_transliteration_map)

In [16]:
up_2021['gp'] = up_2021['gp'].fillna('')
up_2021['gp'] = up_2021['gp'].str.replace('-', ' - ', regex=False)
up_2021[['gp_num', 'gp_name']] = up_2021['gp'].str.split('-', n=1, expand=True)

In [17]:
up_2021['gp_num'] = up_2021['gp_num'].str.strip()
up_2021['gp_name'] = up_2021['gp_name'].str.strip()

In [18]:
up_2021['key'] = up_2021[['district_name', 'block_name', 'gp']].apply(lambda x: '_'.join(x.astype(str)), axis=1)

def assign_winner(group):
    winner_name = group.loc[group['result'] == 'विजेता', 'candidate'].values
    if len(winner_name) > 0:
        group['elected_sarpanch_name'] = winner_name[0]
    else:
        group['elected_sarpanch_name'] = None
    return group

up_2021 = up_2021.groupby('key').apply(assign_winner)

### Load and merge transliterations

In [19]:
up_2005_trans = pd.read_csv("../data/transliteration/up_gp_sarpanch_2005_transliterate_out.csv", low_memory = False)
up_2010_trans = pd.read_csv("../data/transliteration/up_gp_sarpanch_2010_transliterate_out.csv", low_memory = False)

district_translit = pd.read_csv("../data/transliteration/district_official_hindi_english.csv")
block_translit = pd.read_csv("../data/transliteration/block_name_transliteration.csv")

In [20]:
# Let's convert to dict
mapping_dict_1 = up_2005_trans.set_index('Name')['Transliterated'].str.strip().to_dict()
mapping_dict_2 = up_2010_trans.set_index('Name')['Transliterated'].str.strip().to_dict()
mapping_dict_3 = district_translit.set_index('hindi')['eng'].str.strip().to_dict()
mapping_dict_4 = block_translit.set_index('hindi')['eng'].str.strip().to_dict()

merged_dict = reduce(lambda x, y: x | y, [mapping_dict_1, mapping_dict_2, mapping_dict_3, mapping_dict_4])
len(merged_dict)

79628

### Map

In [21]:
up_2005['gp_name_eng'] = up_2005['gp_name_fin'].str.strip().map(merged_dict)
up_2010['gp_name_eng'] = up_2010['gp_name_fin'].str.strip().map(merged_dict)
up_2015['gp_name_eng'] = up_2015['gp_name'].str.strip().map(merged_dict)
up_2021['gp_name_eng'] = up_2021['gp_name'].str.strip().map(merged_dict)

In [22]:
print(up_2005['gp_name_eng'].notna().sum())
print(up_2010['gp_name_eng'].notna().sum())
print(up_2015['gp_name_eng'].notna().sum())
print(up_2021['gp_name_eng'].notna().sum())

50211
21898
39846
253629


In [23]:
up_2005['district_name_eng'] = up_2005['district_name'].str.strip().map(merged_dict)
up_2010['district_name_eng'] = up_2010['district_name'].str.strip().map(merged_dict)
up_2015['district_name_eng'] = up_2015['district_name'].str.strip().map(merged_dict)
up_2021['district_name_eng'] = up_2021['district_name'].str.strip().map(merged_dict)

In [24]:
print(up_2005['district_name_eng'].notna().sum())
print(up_2010['district_name_eng'].notna().sum())
print(up_2015['district_name_eng'].notna().sum())
print(up_2021['district_name_eng'].notna().sum())

51879
51861
56421
329260


In [25]:
up_2005['block_name_eng'] = up_2005['block_name'].str.strip().map(merged_dict)
up_2010['block_name_eng'] = up_2010['block_name'].str.strip().map(merged_dict)
up_2015['block_name_eng'] = up_2015['block_name'].str.strip().map(merged_dict)
up_2021['block_name_eng'] = up_2021['block_name'].str.strip().map(merged_dict)

In [26]:
print(up_2005['block_name_eng'].notna().sum())
print(up_2010['block_name_eng'].notna().sum())
print(up_2015['block_name_eng'].notna().sum())
print(up_2021['block_name_eng'].notna().sum())

51879
51415
54052
341735


In [27]:
up_2005['elected_sarpanch_name_eng'] = up_2005['elected_sarpanch_name'].str.strip().map(merged_dict).str.strip()
up_2010['elected_sarpanch_name_eng'] = up_2010['elected_sarpanch_name'].str.strip().map(merged_dict).str.strip()
up_2015['elected_sarpanch_name_eng'] = up_2015['elected_sarpanch_name'].str.strip().map(merged_dict).str.strip()
up_2021['elected_sarpanch_name_eng'] = up_2021['elected_sarpanch_name'].str.strip().map(merged_dict).str.strip()

In [28]:
print(up_2005['elected_sarpanch_name_eng'].notna().sum())
print(up_2010['elected_sarpanch_name_eng'].notna().sum())
print(up_2015['elected_sarpanch_name_eng'].notna().sum())
print(up_2021['elected_sarpanch_name_eng'].notna().sum())

42690
51688
45034
271036


In [29]:
up_2010['husband_spouse_name_eng'] = up_2010['husband_spouse_name'].str.strip().map(merged_dict).str.strip()
up_2015['husband_spouse_name_eng'] = up_2015['father_husband'].str.strip().map(merged_dict).str.strip()
up_2021['husband_spouse_name_eng'] = up_2021['father_husband'].str.strip().map(merged_dict).str.strip()

In [30]:
up_2005.to_parquet("../data/fin/up_gp_sarpanch_2005_fixed_with_transliteration.parquet", index = False)
up_2010.to_parquet("../data/fin/up_gp_sarpanch_2010_fixed_with_transliteration.parquet", index = False)

# 2015 coercion
columns_to_convert = ['mobile_number', 'valid_votes_received', 'votes_received_percent', 'voting_percent']
up_2015[columns_to_convert] = up_2015[columns_to_convert].astype(str)
up_2015.to_parquet("../data/fin/up_gp_sarpanch_2015_fixed_with_transliteration.parquet", index = False)
up_2021.to_parquet("../data/fin/up_gp_sarpanch_2021_fixed_with_transliteration.parquet", index = False)